In [ ]:
import sys
import numpy
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, multi_gpu_model
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
#Load ascci text and convert to lowercase(this is useful to reduce the vocabulary)
filename = "UsefulPhrases.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [ ]:
#OLD UsefulPhrases
old ={"%","(",")","-","/","[","]","=","0","1","2","3","4","5","6","7","8","9"}
#old = {"@","[","]","/","\n","%","$","#","-","‘","’","*","“","”","(",")"}
#old = {"@","[","]","/","\n","%","$","'","\"","#","-","‘","’","*","“","”","(",")","*","--"}
for old in old:
    raw_text = raw_text.replace(old, "")

In [ ]:
raw_text

In [ ]:
#Create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i,c in enumerate(chars))

In [ ]:
char_to_int

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

In [ ]:
len(dataX)

In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
y[5]

In [ ]:
X[5]

In [ ]:
X.shape, y.shape

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(1000, input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(1000))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model = multi_gpu_model(model, gpus=2)
adm = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adm, metrics=['accuracy'])

In [ ]:
# define the checkpoint
#filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
filepath = "BestModelv4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='auto')

#history = model.fit(X, y, epochs=500, batch_size=X.shape[0]//64, callbacks=[checkpoint], validation_split=0.1)
history = model.fit(X, y, epochs=500, batch_size=X.shape[0]//256, callbacks=[checkpoint], validation_split=0.1)

In [ ]:
# Plot model accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Plot model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
###OPT
from pandas import DataFrame
from time import time

train_loss = DataFrame()
val_loss = DataFrame()
train_acc = DataFrame()
val_acc = DataFrame()
#learning_rate = 0.01, 0.001, 0.0001
#batch_size = 32, 64, 128
mem_unit = 250, 500, 750, 1000
#for learning_rate in learning_rate:
for mem_unit in mem_unit:
    #print('Training on Learning Rate Value set to:', learning_rate)
    #print('Training on Batch Size Value set to:', batch_size)
    print('Training on Batch Size Value set to:', mem_unit)
    # define the LSTM model
    model = Sequential()
    model.add(LSTM(mem_unit, input_shape=(X.shape[1],X.shape[2])))
    model.add(Dropout(0.5))
    model.add(Dense(y.shape[1], activation='softmax'))
    
    model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy'])
    init = time()
    #history = model.fit(X, y, epochs=250, batch_size=X.shape[0]//batch_size, validation_split=0.1, shuffle=False) 
    history = model.fit(X, y, epochs=500, batch_size=X.shape[0]//64, shuffle=False) 
    training_time = time() - init
    print('')
    print('Training time:', training_time)
    print('')
    train_loss[str(mem_unit)] = history.history['loss']
    #val_loss[str(batch_size)] = history.history['val_loss']
    train_acc[str(mem_unit)] = history.history['acc']
    #val_acc[str(batch_size)] = history.history['val_acc']

# plot train and validation loss across multiple runs
plt.plot(train_loss, label='train')
#plt.plot(val_loss, label='validation')
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['Tr_L.R:0.1', 'Tr_L.R:0.01', 'Tr_L.R:0.001', 'Val_L.R:0.1', 'Val_L.R:0.01', 'Val_L.R:0.001'], loc='best')
#plt.legend(['Tr_B:32', 'Tr_B:64', 'Tr_B:128', 'Val_B:32', 'Val_B:64', 'Val_B:128'], loc='best')
#plt.legend(['Tr_B:32', 'Tr_B:64', 'Tr_B:128'], loc='best')
plt.legend(['MemU:250', 'MemU:500', 'MemU:750', 'MemU:1000'], loc='best')
plt.savefig('Bs_Loss_3.png')
plt.show()

# plot train and validation loss across multiple runs
plt.plot(train_acc, label='validation')
#plt.plot(val_acc, label='validation')
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.legend(['Tr_L.R:0.1', 'Tr_L.R:0.01', 'Tr_L.R:0.001', 'Val_L.R:0.1', 'Val_L.R:0.01', 'Val_L.R:0.001'], loc='best')
#plt.legend(['Tr_B:32', 'Tr_B:64', 'Tr_B:128', 'Val_B:32', 'Val_B:64', 'Val_B:128'], loc='best')
#plt.legend(['Tr_B:32', 'Tr_B:64', 'Tr_B:128'], loc='best')
plt.legend(['MemU:250', 'MemU:500', 'MemU:750', 'MemU:1000'], loc='best')
plt.savefig('Bs_Acc_3.png')
plt.show()

In [ ]:
# plot train and validation loss across multiple runs
plt.plot(train_loss, label='train')
#plt.plot(val_loss, label='validation')
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['L.R:0.01', 'L.R:0.001', 'L.R:0.001'], loc='best')
plt.savefig('Compair_Loss.png')
plt.show()

# plot train and validation loss across multiple runs
plt.plot(train_acc, label='train')
#plt.plot(val_acc, label='validation')
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['L.R:0.01', 'L.R:0.001', 'L.R:0.001'], loc='best')
plt.savefig('Compair_Acc.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot model accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'], color='red')
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='best')
plt.savefig('Model_acc.png')
plt.show()

In [ ]:
# Plot model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], color='red')
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='best')
plt.savefig('Model_loss.png')
plt.show()

In [ ]:
model = load_model('/home/abacus/DeepL/RecurrentNN/Project/BestModelv4.hdf5')

In [ ]:
model.summary()

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
#pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(200):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("")
print("\nDone.")